In [29]:
import os
import cv2
import numpy as np
import torch
from PIL import Image, ImageDraw, ImageFont
from facenet_pytorch import InceptionResnetV1, MTCNN
from scipy.spatial.distance import cosine
from datetime import datetime
import time
from collections import defaultdict
import firebase_admin
from firebase_admin import credentials, db
from torchvision import transforms
from google.cloud import storage
from io import BytesIO
import json
import pickle

In [30]:
cred = credentials.Certificate(r"C:\Users\enesb\Documents\GitHub\Real-Time-Face-Recognition-and-Attendance-System\real-time-attendance-sys-13a15-firebase-adminsdk-fbsvc-0b0ea93420.json")
if not firebase_admin._apps:
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://real-time-attendance-sys-13a15-default-rtdb.europe-west1.firebasedatabase.app/'
    })

In [31]:
# Google Cloud Storage Bağlantısı
storage_client = storage.Client.from_service_account_json(
	r"C:\Users\enesb\Documents\GitHub\Real-Time-Face-Recognition-and-Attendance-System\real-time-attendance-460605-4367d4b382a9.json"
)
bucket = storage_client.bucket("dataset-aee")

In [32]:
mtcnn = MTCNN(keep_all=True)
model = InceptionResnetV1(pretrained='vggface2').eval()

In [33]:
# known_faces cache'den yüklenir
if os.path.exists('known_faces.pkl'):
    with open('known_faces.pkl', 'rb') as f:
        known_faces = pickle.load(f)
    print("Cache'den known_faces yüklendi.")
else:
    known_faces = {}

# Daha önce işlenmiş dosyaları yükleme
CACHE_FILE = 'cached_faces.json'
if os.path.exists(CACHE_FILE):
    with open(CACHE_FILE, 'r') as f:
        processed_files = set(json.load(f))
else:
    processed_files = set()

new_processed_files = set()
person_images = {}

blobs = bucket.list_blobs(prefix="dataset/")
for blob in blobs:
    if blob.name.endswith(".jpg") and blob.name not in processed_files:
        path_parts = blob.name.split("/")
        if len(path_parts) >= 3:
            person_name = path_parts[1]
            img_bytes = blob.download_as_bytes()
            img = Image.open(BytesIO(img_bytes)).convert("RGB")

            if person_name not in person_images:
                person_images[person_name] = []
            person_images[person_name].append(img)
            new_processed_files.add(blob.name)

for person, images in person_images.items():
    embeddings = []
    for img in images:
        faces = mtcnn(img)
        if faces is not None:
            face_tensor = faces[0].unsqueeze(0)
            embedding = model(face_tensor).detach().numpy()
            embeddings.append(embedding)
    if embeddings:
        if len(embeddings) == 0:
            print(f"'{person}' için yüz bulunamadı.")
        known_faces[person] = np.mean(embeddings, axis=0)

Cache'den known_faces yüklendi.


In [34]:
# Cache dosyasını güncelle
all_processed = processed_files.union(new_processed_files)
with open(CACHE_FILE, 'w') as f:
    json.dump(list(all_processed), f)

with open('known_faces.pkl', 'wb') as f:
    pickle.dump(known_faces, f)    

print("GCS üzerinden veritabanı oluşturuldu:", known_faces.keys())

GCS üzerinden veritabanı oluşturuldu: dict_keys(['abdullatif_akkaya', 'ahmet_furkan_aylac', 'ahmet_kadir_cicek', 'alper_isleyen', 'alperen_gozum', 'elifsu_karayigit', 'emre_salaman', 'enes_berk_demirci', 'feyzanur_sanri', 'firdevs_eyidogan', 'ghayad_basmah_ji', 'gokhan_guney', 'hatice_aydogan', 'mustafa_kerem_duzdemir', 'oguzhan_cinar', 'rabia_hasim', 'serhat_derya', 'seyma_hacifettahoglu', 'songul_genc', 'tuba_calci', 'zahit_bahadir_camur', 'zehra_akcin', 'uzumlu_kek'])


In [35]:
def show_attendance():
    ref = db.reference('attendance')
    records = ref.get()

    if not records:
        print("\nYoklama verisi bulunamadı.")
        return

    print("\nYoklama Listesi:")
    print("Name\t\t\tTimestamp")
    print("-" * 40)

    sorted_records = sorted(records.values(), key=lambda x: x['timestamp'], reverse=True)
    for record in sorted_records:
        print(f"{record['name']}\t\t{record['timestamp']}")

def compare_faces(embedding, known_faces):
    min_distance = float('inf')
    name = None
    for known_name, known_embedding in known_faces.items():
        distance = cosine(embedding.flatten(), known_embedding.flatten())
        if distance < min_distance:
            min_distance = distance
            name = known_name
    if min_distance < 0.3:
        return name
    else:
        return "Tanımlanamayan Kişi"

recorded_names = set()

def add_attendance(name):
    if name in recorded_names:
        return
    now = datetime.now()
    current_time = now.strftime("%Y-%m-%d %H:%M:%S")
    ref = db.reference('attendance')
    ref.push({
        'name': name,
        'timestamp': current_time,
        'status': 'Present'
    })
    recorded_names.add(name)
    print(f"{name} için yoklama Firebase'e kaydedildi: {current_time}")

verification_counts = defaultdict(int)
verification_threshold = 5

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Kamera açılamadı!")
    exit()

try:
    prev_time = time.time()
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Kare okunamadı!")
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil_frame = Image.fromarray(frame_rgb)

        boxes, _ = mtcnn.detect(pil_frame)
        if boxes is not None:
            draw = ImageDraw.Draw(pil_frame)
            for box in boxes:
                draw.rectangle(box.tolist(), outline=(255, 0, 0), width=3)
                face = pil_frame.crop((box[0], box[1], box[2], box[3]))
                face = face.resize((160, 160))

                face_tensor = transforms.ToTensor()(face).unsqueeze(0)

                face_embedding = model(face_tensor).detach().numpy()

                name = compare_faces(face_embedding, known_faces)

                font = ImageFont.truetype("arial.ttf", 30)
                draw.text((box[0], box[1] - 30), name, fill=(255, 0, 0), font=font)

                if name != "Tanımlanamayan Kişi":
                    verification_counts[name] += 1
                    if verification_counts[name] >= verification_threshold:
                        add_attendance(name)
                        verification_counts[name] = 0
                else:
                    verification_counts = defaultdict(int)

        frame_with_boxes = cv2.cvtColor(np.array(pil_frame), cv2.COLOR_RGB2BGR)

        curr_time = time.time()
        fps = 1 / (curr_time - prev_time)
        prev_time = curr_time

        cv2.putText(frame_with_boxes, f"FPS: {fps:.2f}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        cv2.imshow('Face Recognition', frame_with_boxes)

        if cv2.waitKey(1) & 0xFF == ord('w'):
            break
finally:
    cap.release()
    cv2.destroyAllWindows()

enes_berk_demirci için yoklama Firebase'e kaydedildi: 2025-05-22 17:52:06
